In [ ]:
#3周真是数据 预测第一周3周平均  预测第二周 两周真实+ 一周预测

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import datetime
import time
import matplotlib
import base
import predict
#import holiday_tread as tread

%load_ext autoreload
%autoreload 2

In [2]:
#读取user_pay_count
user_pay_counts = base.ReadUserPayCountCSV();
#读取开店时间数据
shop_open_dates = base.ReadShopOpenDateCSV();
###读取商家信息
shop_infos = base.ReadShopInfoCSV();
##读取城市名称对照表
city_names = base.ReadCityNameCSV();
#读取休假表
calenders = base.ReadCalendarCSV();

In [39]:
def initWeekDate(start_time, week_num):
    time_ranges = []
    while(week_num > 0):
        end_time = start_time + datetime.timedelta(days=6)
        time_ranges.append([start_time, end_time])

        week_num = week_num - 1;
        start_time = start_time + datetime.timedelta(weeks=1)

    return time_ranges

In [40]:
def trainSingleHolidayMeanData(id, shop_open_dates, user_pay_counts, calenders, start_time, end_time):
    open_time = shop_open_dates.loc[id]['date'];
    if open_time > start_time:
        start_time = open_time
    df = base.countShopPayTimePeriods(user_pay_counts, id,
                                      date_range=[start_time, end_time], time_range=[datetime.timedelta(hours=0), datetime.timedelta(hours=23)])

    df['holiday'] = calenders.loc[df.index.strftime('%Y-%m-%d')]['daytype'].values

    means = {}
    for i in range(1, 6):
        if i != 4:
            temp_df = df[df['holiday'] == i];
            means[i] = temp_df['count'].mean()

    #print(mean_dict)
    #result_df = pd.DataFrame(mean_dict).T
    #result_df.rename(columns={0: 'mean', 1: 'std'}, inplace=True)

    return means


def trainHolidayMeansData(shop_open_dates, user_pay_counts, calenders, time_ranges):
    shop_mean_dict = {}
    for id in range(1,2001):
        print(id)
        mean_dict = {}
        for i, tr in enumerate(time_ranges):
            means = trainSingleHolidayMeanData(id, shop_open_dates, user_pay_counts, calenders, tr[0], tr[1])
            mean_dict[i] = means
        
        shop_mean_dict[id] = mean_dict;

    return shop_mean_dict;

In [41]:
def getHolidayMeanCount(id, day_type, shop_mean_dict, start_real, end_real, pred_mean=None):
    count = 0;
    for i in range(start_real, end_real):
        count = count + shop_mean_dict[id][i][day_type]
    
    if pred_mean != None:
        count = count + pred_mean[id][day_type]
            
    if pred_mean != None:
        return count / (end_real - start_real  + 1);
    else:
        return count / (end_real - start_real)

In [42]:
week_num = 3
time_ranges = initWeekDate(pd.to_datetime('2016-10-11'), week_num)
shop_mean_dict = trainHolidayMeansData(shop_open_dates, user_pay_counts, calenders, time_ranges)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [43]:
shop_mean_dict

{1: {0: {1: 232.0, 2: 244.0, 3: 264.0, 5: 264.0},
  1: {1: 211.0, 2: 230.0, 3: 202.0, 5: 243.0},
  2: {1: 208.25, 2: 257.0, 3: 251.0, 5: 281.0}},
 2: {0: {1: 0.0, 2: 0.0, 3: 0.0, 5: 0.0},
  1: {1: 19.5, 2: 61.0, 3: 155.0, 5: 143.0},
  2: {1: 85.5, 2: 96.0, 3: 96.0, 5: 115.0}},
 3: {0: {1: 65.5, 2: 71.0, 3: 119.0, 5: 97.0},
  1: {1: 66.25, 2: 103.0, 3: 127.0, 5: 125.0},
  2: {1: 69.0, 2: 103.0, 3: 102.0, 5: 104.0}},
 4: {0: {1: 89.75, 2: 121.0, 3: 165.0, 5: 200.0},
  1: {1: 85.5, 2: 80.0, 3: 211.0, 5: 221.0},
  2: {1: 91.0, 2: 110.0, 3: 215.0, 5: 196.0}},
 5: {0: {1: 236.25, 2: 195.0, 3: 185.0, 5: 211.0},
  1: {1: 271.5, 2: 187.0, 3: 161.0, 5: 240.0},
  2: {1: 135.5, 2: 215.0, 3: 203.0, 5: 291.0}},
 6: {0: {1: 86.75, 2: 121.0, 3: 112.0, 5: 137.0},
  1: {1: 100.5, 2: 107.0, 3: 88.0, 5: 76.0},
  2: {1: 100.0, 2: 133.0, 3: 109.0, 5: 118.0}},
 7: {0: {1: 123.0, 2: 140.0, 3: 183.0, 5: 208.0},
  1: {1: 108.0, 2: 132.0, 3: 152.0, 5: 200.0},
  2: {1: 108.75, 2: 128.0, 3: 165.0, 5: 162.0}},
 8: 

In [44]:
start_time = pd.to_datetime('2016-11-1')
end_time = pd.to_datetime('2016-11-7')
predict_date = pd.date_range(start=start_time, end=end_time, freq='D', normalize=True)
pridicts = {}
shop_pred_dict = {}
for id in range(1, 2001):
    print(id)
    y_Pred = []
    pred_dict = {}
    for date in predict_date:
        daytype = calenders.loc[date.strftime('%Y-%m-%d')]['daytype']
        count = getHolidayMeanCount(id, daytype, shop_mean_dict, 0, 3)
        y_Pred.append(count);
        pred_dict[daytype] = count;
    
    shop_pred_dict[id] = pred_dict;
    pridicts[id] = y_Pred


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [45]:
start_time = pd.to_datetime('2016-11-8')
end_time = pd.to_datetime('2016-11-14')
predict_date = pd.date_range(start=start_time, end=end_time, freq='D', normalize=True)
for id in range(1, 2001):
    print(id)
    y_Pred = []
    for date in predict_date:
        daytype = calenders.loc[date.strftime('%Y-%m-%d')]['daytype']
        count = getHolidayMeanCount(id, daytype, shop_mean_dict, 1, 3, shop_pred_dict)
        pridicts[id].append(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [46]:
predict_df = pd.DataFrame(pridicts)
predict_df = predict_df.T
predict_df = predict.FixFinalPredict(predict_df)

In [47]:
predict_df = predict.FixFinalPredict(predict_df)
predict_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
1,217,217,217,243,239,262,217,212,212,212,243,230,262,212
2,35,35,35,52,83,86,35,46,46,46,69,111,114,46
3,66,66,66,92,116,108,66,67,67,67,99,115,112,67
4,88,88,88,103,197,205,88,88,88,88,97,207,207,88
5,214,214,214,199,183,247,214,207,207,207,200,182,259,207
6,95,95,95,120,103,110,95,98,98,98,120,100,101,98
7,113,113,113,133,166,190,113,110,110,110,131,161,184,110
8,63,63,63,48,80,87,63,62,62,62,47,72,82,62
9,216,216,216,203,254,291,216,216,216,216,205,255,294,216
10,97,97,97,85,89,103,97,97,97,97,84,93,103,97


In [48]:
predict_df.to_csv("./data/dataset/dataset/predict/prediction.csv", header = None, index=True, encoding='UTF-8')